# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [2]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

IMAGE_SIZE = 28




1.   **Normalization:** Add an extra transformation for normalizing the data because we are using
  tanh as an activation function (range -1, 1)
2.   **Image resizing:** Standarize all images size to 28x28
3. **Missing activation:** Adding a tanh activation in the generator
4. **Z reshape**: the generator takes input z 1x100, so we need to reshape the input to 100 instead of 64
5. **Double use of sigmoid:** We are using BCE with logits as our loss function, which internally use sigmoid. Therefore, we can remove the last sigmoid layer in the discriminator
6. **-1 in view():**:    "return self.net(x).view(x.size(0), -1)"
7. TODO: FIX TRAINING LOGIC




In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform=transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)), # (input - mean) / std. (Single channel)

    ])
loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 509kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.66MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.24MB/s]


In [4]:
images, labels = next(iter(loader))
print('Image shape:', images.shape)
print('Label shape:', labels.shape)
print('First label:', labels[0])


Image shape: torch.Size([256, 1, 28, 28])
Label shape: torch.Size([256])
First label: tensor(3)


In [5]:
z_dim = 100
g_lr = 2e-2
d_lr = 2e-5

class D(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 1, 7, 1, 0)
        )

    def forward(self, x):
        return self.net(x).view(x.size(0), -1)

class G(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(100, 128, 4, 1, 0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), 100, 1, 1)
        return self.net(z)

Dnet = D().to(device)
Gnet = G().to(device)

crit = nn.BCEWithLogitsLoss()
opt_d = torch.optim.Adam(Dnet.parameters(), lr=d_lr, betas=(0.9, 0.999))
opt_g = torch.optim.Adam(Gnet.parameters(), lr=g_lr, betas=(0.9, 0.999))

In [ ]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device); b=real.size(0)
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_d=crit(Dnet(real),torch.zeros(b,1,device=device))+crit(Dnet(fake),torch.ones(b,1,device=device))  # BUG labels
    loss_d.backward(); opt_g.step()  # BUG
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_g=crit(Dnet(fake),torch.zeros(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
print('Now fix all the issues.')